This is a simplified implementation of the GradientDescentOptimizer in tensorflow using general backpropagation. Code is referenced from the Tensorflow source code, specifically, `gradients.py` and `math_grad.py`

In [ ]:
# Import data
from tensorflow.examples.tutorials.mnist import input_data

import tensorflow as tf

flags = tf.app.flags
FLAGS = flags.FLAGS
flags.DEFINE_string('data_dir', '/tmp/data/', 'Directory for storing data')

In [14]:
from tensorflow.python.ops import gen_array_ops
from sklearn.metrics import accuracy_score
import numpy as np
from collections import deque

In [3]:
sess = tf.InteractiveSession()

In [4]:
mnist = input_data.read_data_sets(FLAGS.data_dir, one_hot=True)

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [17]:
class NeuralNet:
    def __init__(self, epoch=1000, learning_rate=0.5, batch_size=100):
        self.epoch = epoch
        self.learning_rate = learning_rate
        self.batch_size = batch_size
        self.W1 = None
        self.b1 = None
    
    def fit(self, train, use_tf=False):
        batch_X, batch_y = train.next_batch(self.batch_size)
        d = batch_X.shape[1]
        c = batch_y.shape[1]

        tf_X = tf.placeholder(tf.float32, [None, d])
        tf_y = tf.placeholder(tf.float32, [None, c])
        self.W1 = tf.Variable(tf.random_normal([d, c], seed=1)/d)
        self.b1 = tf.Variable(tf.zeros(c))
        z = tf.matmul(tf_X, self.W1) + self.b1
        y_hat = tf.nn.softmax(z)
        cross_entropy = tf.reduce_mean(-tf.reduce_sum(tf_y * tf.log(y_hat), reduction_indices=[1]))

        sess.run(tf.initialize_variables([self.W1, self.b1]))

        if use_tf:
            steps = tf.train.GradientDescentOptimizer(self.learning_rate).minimize(cross_entropy)
        else:
            steps = self.train_step(cross_entropy)

        for i in range(self.epoch):
            if i != 0:
                batch_X, batch_y = train.next_batch(self.batch_size)
            sess.run(steps, feed_dict={tf_X: batch_X, tf_y: batch_y})

    def predict(self, X):
        tf_X = tf.placeholder(tf.float32, [None, X.shape[1]])
        y_hat = tf.nn.softmax(tf.matmul(tf_X, self.W1) + self.b1)

        return sess.run(y_hat, feed_dict={tf_X: X})

    def var_list(self):
        return [self.W1, self.b1]

    def train_step(self, cost):
        grads_and_vars = self.gradients(cost)
        return [tf.assign(var, var - self.learning_rate * grad) for grad, var in grads_and_vars]

    def gradients(self, cost):
        ''' Ref: Deep Learning Book: 6.5.6 General back-propagation; Tensorflow paper: 4.1 Gradient computation
        
        Starting from cost output, use chain rule to back propagate gradients to variables.
        '''
        op = cost.op
        # grads store the output gradient of an operation
        grads = {op: self.default_gradient(op)}
        queue = deque()
        self._append_op_to_queue(queue, op)
        while queue:
            op = queue.popleft()
            # extract the gradient function to calculate the input gradients by applying chain rule
            grad_fn = self._grad_fn(op)
            in_grads = grad_fn(op, grads[op])
            for t_in, in_grad in zip(op.inputs, in_grads):
                grads[t_in.op] = in_grad
                self._append_op_to_queue(queue, t_in.op)

        return [(grads[xs.op], xs) for xs in self.var_list()]

    def default_gradient(self, op):
        out = op.outputs[0]
        return tf.fill(tf.shape(out), tf.constant(1, dtype=out.dtype))

    def _append_op_to_queue(self, queue, op):
        if len(op.inputs) > 0:
            queue.append(op)

    def _grad_fn(self, op):
        if op.type == 'MatMul':
            return self._matmul_grad
        elif op.type == 'Add':
            return self._add_grad
        elif op.type == 'Identity':
            return self._id_grad
        elif op.type == 'Softmax':
            return self._softmax_grad
        elif op.type == 'Log':
            return self._log_grad
        elif op.type == 'Mul':
            return self._mul_grad
        elif op.type == 'Sum':
            return self._sum_grad
        elif op.type == 'Neg':
            return self._neg_grad
        elif op.type == 'Mean':
            return self._mean_grad

    def _add_grad(self, op, grad):
        x = op.inputs[0]
        y = op.inputs[1]
        sx = tf.shape(x)
        sy = tf.shape(y)
        rx, ry = gen_array_ops._broadcast_gradient_args(sx, sy)
        return (
            tf.reshape(tf.reduce_sum(grad, rx), sx),
            tf.reshape(tf.reduce_sum(grad, ry), sy)
        )

    def _matmul_grad(self, op, grad):
        x = op.inputs[0]
        y = op.inputs[1]
        return (tf.matmul(grad, y, transpose_b=True), tf.matmul(x, grad, transpose_a=True))

    def _id_grad(self, op, grad):
        return (grad,)

    def _softmax_grad(self, op, grad):
        softmax = op.outputs[0]
        return ((grad - tf.reshape(tf.reduce_sum(grad * softmax, [1]), [-1, 1])) * softmax,)

    def _log_grad(self, op, grad):
        x = op.inputs[0]
        return (grad * tf.inv(x),)

    def _mul_grad(self, op, grad):
        x = op.inputs[0]
        y = op.inputs[1]
        sx = tf.shape(x)
        sy = tf.shape(y)
        rx, ry = gen_array_ops._broadcast_gradient_args(sx, sy)
        return (
            tf.reshape(tf.reduce_sum(grad * y, rx), sx),
            tf.reshape(tf.reduce_sum(grad * x, ry), sy)
        )

    def _sum_grad(self, op, grad):
        input_shape = tf.shape(op.inputs[0])
        axes = op.inputs[1]
        reduced_shape = self._reduced_shape(input_shape, axes)
        tile_scaling = input_shape // reduced_shape
        return (tf.tile(tf.reshape(grad, reduced_shape), tile_scaling),)

    def _neg_grad(self, op, grad):
        return (-grad,)

    def _mean_grad(self, op, grad):
        sum_grad = self._sum_grad(op, grad)[0]
        input_shape = tf.shape(op.inputs[0])
        output_shape = tf.shape(op.outputs[0])
        factor = tf.reduce_prod(input_shape) // tf.reduce_prod(output_shape)
        return (sum_grad / tf.cast(factor, sum_grad.dtype),)

    def _reduced_shape(self, input_shape, axes):
        '''
        For example: tf.reduce_sum(input, axes): input shape [2, 5, 3, 7], axes = [1, 2]
        Then reduced_shape is [2, 1, 1, 7]
        '''
        input_rank = tf.size(input_shape)
        axes = (axes + input_rank) % input_rank
        return tf.dynamic_stitch(
            [tf.range(input_rank), axes],
            [input_shape, tf.fill(tf.shape(axes), 1)]
        )

## Tensorflow General Back Propagation

In [18]:
nn = NeuralNet(epoch=1000, batch_size=100, learning_rate=0.5)
nn.fit(mnist.train, use_tf=True)

In [19]:
y_hat = nn.predict(mnist.test.images)
accuracy_score(np.argmax(mnist.test.labels, 1), np.argmax(y_hat, 1))

0.91620000000000001

## Manual General Back Propagation

In [20]:
nn = NeuralNet(epoch=1000, batch_size=100, learning_rate=0.5)
nn.fit(mnist.train, use_tf=False)

y_hat = nn.predict(mnist.test.images)
accuracy_score(np.argmax(mnist.test.labels, 1), np.argmax(y_hat, 1))

0.91790000000000005